In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [9]:
import time

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)
bs = 1
epochs = 1


Using cpu device
Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100.0%


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100.0%


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100.0%


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1000)>



100.0%

Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw



In [65]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 10),
            nn.ReLU(),
            nn.Linear(10, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
        
def get_model():
    model = NeuralNetwork()
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
    return model, loss_fn, optimizer

def get_data(training_data, test_data, bs):
    return (
        DataLoader(training_data, batch_size=bs), #, shuffle=True),
        DataLoader(test_data, batch_size=bs),
    )

def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    correct = 0
    size = len(train_dl.dataset)
    for epoch in range(epochs):
        model.train()
        for batch, (xb, yb) in enumerate(train_dl):
            # xb = xb.to(device), yb.to(device)
            loss, _ = loss_batch(model, loss_func, xb, yb, opt)
            correct += bool(model(xb).argmax() == yb)
            
            if batch % 100 == 0:
                current = (batch + 1) * len(xb)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        

        correct /= len(train_dl.dataset)

        print(f"Training Error: \n Accuracy: {(100*correct):>0.1f}%")


In [66]:
epochs = 10
train_dl, valid_dl = get_data(training_data, test_data, bs)

In [13]:
torch.manual_seed(0)
train_dl, valid_dl = get_data(training_data, test_data, bs)
model, loss_func, opt = get_model()
torch.save(model.state_dict(), "model.pth")
start_time = time.time()
fit(epochs, model, loss_func, opt, train_dl, valid_dl)
stop_time = time.time()
print(stop_time-start_time)

loss: 2.399653  [    1/60000]
loss: 2.330267  [ 1001/60000]
loss: 2.712936  [ 2001/60000]
loss: 2.357742  [ 3001/60000]
loss: 1.477537  [ 4001/60000]
loss: 1.230931  [ 5001/60000]
loss: 0.831079  [ 6001/60000]
loss: 1.956707  [ 7001/60000]
loss: 0.114509  [ 8001/60000]
loss: 0.734352  [ 9001/60000]
loss: 0.196807  [10001/60000]
loss: 0.086218  [11001/60000]
loss: 0.363267  [12001/60000]
loss: 0.556539  [13001/60000]
loss: 0.114997  [14001/60000]
loss: 0.867073  [15001/60000]
loss: 0.062347  [16001/60000]
loss: 0.333005  [17001/60000]
loss: 0.211893  [18001/60000]
loss: 0.513849  [19001/60000]
loss: 0.008754  [20001/60000]
loss: 0.783907  [21001/60000]
loss: 0.189741  [22001/60000]
loss: 0.011704  [23001/60000]
loss: 0.214129  [24001/60000]
loss: 1.446226  [25001/60000]
loss: 0.262589  [26001/60000]
loss: 0.837122  [27001/60000]
loss: 0.044474  [28001/60000]
loss: 0.127537  [29001/60000]
loss: 0.009463  [30001/60000]
loss: 1.633116  [31001/60000]
loss: 0.118802  [32001/60000]
loss: 1.21

In [68]:
model(xb)

tensor([[ 0.6209, -6.4523,  0.8480,  3.7399, -2.7591, -0.7681, -7.3642, 10.1244,
          0.2005,  4.4697],
        [ 6.9891, -0.9531,  9.2795,  5.8925, -9.3502,  5.8432,  6.4225, -9.6935,
          5.3983, -5.3377]], grad_fn=<AddmmBackward0>)

In [67]:
len(valid_dl)

10000

In [69]:
debug = iter(valid_dl)
next(debug)[0].shape

torch.Size([1, 1, 28, 28])

In [79]:
correct = 0
start_time = time.time()
times = np.zeros(10)
for i in range(10):
    for batch, (xb, yb) in enumerate(valid_dl):
        pred = model(xb).argmax()
        if batch % 1000 == 0:
            print(f"Complete: {batch/len(valid_dl)+i}", end="\r")
        if pred == yb:
            correct +=1
    stop_time = time.time()
    times[i] = stop_time - start_time
print(np.diff(times).mean())


    

IndexError: index 10 is out of bounds for axis 0 with size 10

In [82]:
np.diff(times).mean()

2.7625641557905407